# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

C:\ProgramData\miniconda3\envs\hmj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [3]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(dataset["train"])

,label,text
0,1 star,This business is closed now. This business is closed now. This business is closed now. This business is closed now. This business is closed now. This business is closed now. This business is closed now.
1,4 stars,"This location is a little different than the Verizon stores I'm used to. No check-in counter, no list of people waiting so you can see where you stand, you have to have an employee check you in, and then they will call you when it is your turn. \n\nThe sales staff on the floor seemed disinterested in fixing our tech support issue, instead wanting to instead sell us an upgrade. The sales manager, though, was more than helpful! He completely solved our problem, and turned two unhappy customers into satisfied ones."
2,2 star,"Really good shawarma, Horrible service."
3,1 star,"I would give it a zero if I could! We wanted to get some pizza before we headed out to the clubs so we ordered in and got it delivered. we ordered 3 pizzas and it was quite expensive so I was expecting some darn good pizza. however it definitely fell short. it was horrible! cold, crust was hard, all the toppings would fall off. everything was just horrible. I don't see what people like about their pizza because dominos tastes better than them!"
4,4 stars,"Cool place, good music and great sandwiches at reasonable prices. Was a regular at the one in Boone and while this one seems a little, um, fake I guess it's still okay. The Ricochet Rabbit is one of my favorite sandwiches."
5,4 stars,"A few of my Spanish friends were grilling out last weekend and we had to pick up some meat, charcoal, and veggies to cook. I headed in the direction of Food Lion and Jose navigated me to The Atlantic Farmers Market. It is mostly a Spanish grocery, but it is so worth checking out. If I ever wanted to play Mexican for a night I would definitely go here for tortillas, avocado, and salsa. It is so authentic. When we arrived at the deli, Jose began to tell the butcher exactly what we wanted in Spanish, but just to see, I asked for a few things in English. Turns out most of the bilingual so there is no real language gap. I really enjoyed this culturally enhancing experience and so will you."
6,1 star,"Seriously this is one of the worst Asian food restaurant ever. We were craving pho nearby our hotel so we went despite the fairly positive reviews, but we were so wrong. I had a order of egg rolls which tasted and smelled wrong. The oil they fried these in must've been bad or old. I had never tasted anything that horrible before. I almost wanted to leave the place but waited for my beef pho which I must admit was the worst ever! The soup was bland, the beef smelled and tasted old, there was only green onions as garnish, no cilantro, bean sprouts, basil, whatsoever! My friends ordered Pat Thai, Kung Pao Chicken, Black a Bean Chicken, etc. but they were all just horrible! The only thing that tasted good was the Sriracha..."
7,4 stars,"Hobby Lobby is every crafter's fantasy! Like others have mentioned, this place has it all! It's hard to not spend hours upon hours browsing their inventory imagining what you can create next. \n\nI took my niece here and we had a blast picking out things to make together. I was very impressed with this location because it was well organized, clean, and the aisles were roomy. I was equally impressed with their fabric selection. \n\nThey have great sales each week! The only drawback is that they still hand key every item. Having said that, they do a good job training their employees to type fast and to be knowledgeable on the weekly deals. The checkout process didn't take too long, like other Hobby Lobby's I've been to."
8,5 stars,"Love love Nordstorm Cafe. Fairly cheap prices for quality food. But man does it get busy.\n\nThere's a fairly wide variety of things to choose from, from 8-10 salads, 3 pizza options, to 3 pasta options, and 5-6 sandwich items, as well as a variety of desserts to chose from. \n\nFor bargain deals, get

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

C:\ProgramData\miniconda3\envs\hmj\Lib\site-packages\transformers\utils\hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [12]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,4 stars,"Love the place!! This exactly whats needed after a long day exploring the strip & shopping. \nStaff are super friendly & overall the place is aesthetically pleasing. Usually pretty busy so you may end up waiting a while but the foods worth it. \n\nGot some buffalo chicken strips & guh... Big portions, yet so damned tasty ... & I still don't like celery regardless of what its dipped in. \nRefreshing smoothies to boot with care taken in regards to allergies. \nTried the vanilla bean cheesecake, it was lovely & light. I'm gutted we didn't get to try all of the cakes the bakery looked amazing!!!!! \nReasonably priced as well & overall pleasant experience, so I have no complaints asides from potentially long waits.","[101, 2185, 1103, 1282, 106, 106, 1188, 2839, 1184, 1116, 1834, 1170, 170, 1263, 1285, 12138, 1103, 6322, 111, 6001, 119, 165, 183, 1708, 1777, 3101, 1132, 7688, 4931, 111, 2905, 1103, 1282, 1110, 13492, 2716, 16150, 4253, 119, 12378, 2785, 5116, 1177, 1128, 1336, 1322, 1146, 2613, 170, 1229, 1133, 1103, 11785, 3869, 1122, 119, 165, 183, 165, 183, 2349, 3329, 1199, 26700, 9323, 13507, 111, 176, 23698, 119, 119, 119, 2562, 8924, 117, 1870, 1177, 10688, 27629, 13913, 119, 119, 119, 111, 146, 1253, 1274, 112, 189, 1176, 172, 11194, 1616, 8334, 1104, 1184, 1157, 13916, 1107, 119, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [11]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors: 100%|█████████████████████████████████████████████████████████████| 436M/436M [08:22<00:00, 867kB/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [13]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [14]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [17]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [19]:
from transformers import TrainingArguments, Trainer

#training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer", evaluation_strategy="epoch")

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.412400,1.149103,0.511000
2,1.063300,1.057826,0.545000
3,0.826600,1.021762,0.579000


TrainOutput(global_step=375, training_loss=1.0267659200032553, metrics={'train_runtime': 167.8596, 'train_samples_per_second': 17.872, 'train_steps_per_second': 2.234, 'total_flos': 789354427392000.0, 'train_loss': 1.0267659200032553, 'epoch': 3.0})

In [28]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(200))

In [29]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.0513006448745728,
 'eval_accuracy': 0.51,
 'eval_runtime': 3.503,
 'eval_samples_per_second': 57.095,
 'eval_steps_per_second': 7.137,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [30]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [32]:
trainer.save_state()

家庭作业-HOMEWork 

#以下全数据量训练

In [5]:
#数据预处理
small_train_dataset2 = tokenized_datasets["train"].shuffle(seed=36)
small_eval_dataset2 = tokenized_datasets["test"].shuffle(seed=36)

In [6]:
from transformers import AutoModelForSequenceClassification

model2 = AutoModelForSequenceClassification.from_pretrained("models/bert-base-cased/finetuned-trainer", num_labels=5)

In [7]:
from transformers import TrainingArguments, Trainer

model_dir = "models/bert-base-cased"

'''
evaluation_strategy, 有三种取值
    “no”, 默认取值, 训练期间不 eval
    “steps”, 按照另一个参数 “eval_steps” 指定的步数间隔作评估
    “epoch”, 每个 epoch 结束做一次评估
'''

training_args2 = TrainingArguments(output_dir=f"{model_dir}/test_trainer2",
                                   per_device_eval_batch_size=50,per_device_train_batch_size=13,
                                  evaluation_strategy="steps",eval_steps=10000,
                                  #evaluation_strategy="epoch",
                                  num_train_epochs=2.0,
                                  save_strategy="steps",save_steps=10000,
                                  logging_dir=f"{model_dir}/test_trainer2/runs",
                                  logging_steps=500)


In [8]:
print(training_args2)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=10000,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_bett

In [9]:
#验证方法
import numpy as np
import evaluate
metric2 = evaluate.load("accuracy")

def compute_metrics2(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric2.compute(predictions=predictions, references=labels)


In [10]:
trainer2 = Trainer(
    model=model2,
    args=training_args2,
    train_dataset=small_train_dataset2,
    eval_dataset=small_eval_dataset2,
    compute_metrics=compute_metrics2,
)

In [11]:
#训练开始
trainer2.train()

Step,Training Loss,Validation Loss,Accuracy
10000,0.865600,0.822764,0.641620
20000,0.826100,0.811897,0.643180
30000,0.791100,0.775904,0.662640
40000,0.757300,0.801821,0.656400
50000,0.777700,0.757012,0.668260
60000,0.716200,0.755280,0.675660
70000,0.711100,0.755496,0.675920
80000,0.697500,0.727333,0.685560
90000,0.678200,0.721342,0.689740
100000,0.676000,0.715413,0.691060


TrainOutput(global_step=100000, training_loss=0.7596662670898437, metrics={'train_runtime': 54603.2387, 'train_samples_per_second': 23.808, 'train_steps_per_second': 1.831, 'total_flos': 3.420535852032e+17, 'train_loss': 0.7596662670898437, 'epoch': 2.0})

In [12]:
#最终验证
small_test_dataset2 = tokenized_datasets["test"].shuffle(seed=64).select(range(200))
trainer2.evaluate(small_test_dataset2)

{'eval_loss': 0.7575817704200745,
 'eval_accuracy': 0.685,
 'eval_runtime': 2.9125,
 'eval_samples_per_second': 68.67,
 'eval_steps_per_second': 1.373,
 'epoch': 2.0}

In [14]:
#模型保存
trainer2.save_model(f"{model_dir}/finetuned-trainer2/checkpoint-end/")